In [1]:
%pwd

'/Users/aflamant/Documents/courses/2024-2025/mémoire/03-code/memoire/MLP/experiments/01-single_ea'

In [2]:
%cd ../..

/Users/aflamant/Documents/courses/2024-2025/mémoire/03-code/memoire/MLP


# Import libraries

In [3]:
RANDOM_STATE = 42
import numpy as np

import os
import torch
from torch import nn
from torch.utils.data import random_split, DataLoader, Subset
from sklearn.model_selection import KFold
from MLP.dataset import (TenBarsCantileverTrussSingleEADataset,
                     TwoBarsTrussSingleEADataset,
                     BiSupportedTrussBeamSingleEADataset)
from torch.utils.tensorboard import SummaryWriter

from MLP.models.architecture import MultiLayerPerceptron
from MLP.models.processing import StandardScaler
from MLP.loss import StiffnessToDisplacementLoss, StiffnessToLoadLoss, construct_k_from_ea

np.random.seed(RANDOM_STATE)

ModuleNotFoundError: No module named 'tensorboard'

In [4]:
device = torch.device(
    'cuda' if torch.cuda.is_available()
    else 'mps' if torch.mps.is_available()
    else 'cpu'
)

In [5]:
c1 = StiffnessToLoadLoss()
c2 = StiffnessToDisplacementLoss()

# Data

In [6]:
connectivity = {}
connectivity['cantilever'] = torch.tensor([[0, 1],
                                           [1, 2],
                                           [3, 4],
                                           [4, 5],
                                           [1, 4],
                                           [2, 5],
                                           [0, 4],
                                           [3, 1],
                                           [1, 5],
                                           [4, 2]]).to(device)
connectivity['triangle'] = torch.tensor([[0, 1],
                                         [1, 2]]).to(device)
connectivity['beam'] = torch.tensor([[0, 1],
                                     [1, 2],
                                     [2, 3],
                                     [3, 4],
                                     [5, 6],
                                     [6, 7],
                                     [7, 8],
                                     [8, 9],
                                     [0, 5],
                                     [1, 6],
                                     [2, 7],
                                     [3, 8],
                                     [4, 9],
                                     [0, 6],
                                     [5, 1],
                                     [1, 7],
                                     [6, 2],
                                     [2, 8],
                                     [7, 3],
                                     [3, 9],
                                     [8, 4]]).to(device)

In [7]:
support = {}
support['cantilever'] = torch.tensor([0, 1, 6, 7]).to(device)
support['triangle'] = torch.tensor([0, 1, 4, 5]).to(device)
support['beam'] = torch.tensor([10, 11, 18, 19]).to(device)

In [8]:
names = ['beam', 'cantilever', 'triangle']

path = {
    'beam': 'data/dataset/beam/data.hdf5',
    'cantilever': 'data/dataset/cantilever/data.hdf5',
    'triangle': 'data/dataset/triangle/data.hdf5'
}

_dataset = {
    'beam': BiSupportedTrussBeamSingleEADataset(path['beam']),
    'cantilever': TenBarsCantileverTrussSingleEADataset(path['cantilever']),
    'triangle': TwoBarsTrussSingleEADataset(path['triangle'])
}

In [9]:
_train_ds = {}
_test_ds = {}

for typology in names:
    train_ds, test_ds = random_split(_dataset[typology], (0.8, 0.2))
    _train_ds[typology] = train_ds
    _test_ds[typology] = test_ds

# Model

In [15]:
def main(
        tb_writer,
        dataset,
        elems,
        supports,
        out_size=1,
        hidden_size=70,
        n_hidden=3,
        activation=nn.ReLU,
        activation_params={},
        learning_rate=1e-3,
        n_folds=3,
        n_epochs=30,
):
    in_size = len(dataset[0][0])
    N_EPOCHS = n_epochs
    BATCH_SIZE = 2048  # Not an hyperparameter
    LEARNING_RATE = learning_rate

    train_losses_fold = [[] for _ in range(n_folds)]
    train_data_losses_fold = [[] for _ in range(n_folds)]
    train_physics_losses_fold = [[] for _ in range(n_folds)]

    val_losses_fold = [[] for _ in range(n_folds)]
    val_data_losses_fold = [[] for _ in range(n_folds)]
    val_physics_losses_fold = [[] for _ in range(n_folds)]

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=RANDOM_STATE)
    for fold, (train_indices, test_indices) in enumerate(kf.split(dataset)):
        # Dataset and Dataloader
        train_ds = Subset(dataset, train_indices)
        test_ds = Subset(dataset, test_indices)

        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, )
        val_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, )

        # Training scaler
        x_scaler = StandardScaler(in_size).to(device)
        y_scaler = StandardScaler(out_size).to(device)
        for x, y, _, _, _ in train_loader:
            x_scaler.partial_fit(x.to(device))
            y_scaler.partial_fit(y.to(device))

        # Model setup
        model = MultiLayerPerceptron(in_size, out_size, hidden_size, n_hidden, activation,
                                     activation_params).to(device)

        data_criterion = nn.MSELoss()
        physics_criterion = StiffnessToLoadLoss()

        physics_loss_scale = None

        optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

        # Model training
        for epoch in range(N_EPOCHS):
            # Train
            model.train()

            total_loss = 0.0
            total_data_loss = 0.0
            total_physics_loss = 0.0

            for batch in train_loader:
                data, ea, nodes, u, q = batch

                data, ea = data.to(device), ea.to(device)
                nodes, u, q = nodes.to(device), u.to(device), q.to(device)

                z_data = x_scaler.transform(data)
                z_ea = y_scaler.transform(ea)

                optimizer.zero_grad()
                z_pred_ea = model(z_data)

                ea_pred = y_scaler.inverse_transform(z_pred_ea)
                stiffness_pred = construct_k_from_ea(ea_pred, nodes, elems, supports, device=device)

                data_loss = data_criterion(z_pred_ea, z_ea)
                physics_loss = physics_criterion(stiffness_pred, u, q)

                if physics_loss_scale is None: physics_loss_scale = physics_loss.item()

                physics_loss /= physics_loss_scale

                loss = data_loss + physics_loss
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                total_data_loss += data_loss.item()
                total_physics_loss += physics_loss.item()

            # Log
            train_loss = total_loss / len(train_loader)
            train_physics_loss = total_physics_loss / len(train_loader)
            train_data_loss = total_data_loss / len(train_loader)

            train_losses_fold[fold].append(train_loss)
            train_physics_losses_fold[fold].append(train_physics_loss)
            train_data_losses_fold[fold].append(train_data_loss)

            # Validate
            model.eval()
            total_loss = 0.0
            total_data_loss = 0.0
            total_physics_loss = 0.0
            with torch.no_grad():
                for batch in val_loader:
                    data, ea, nodes, u, q = batch

                    data, ea = data.to(device), ea.to(device)
                    nodes, u, q = nodes.to(device), u.to(device), q.to(device)

                    z_data = x_scaler.transform(data)
                    z_ea = y_scaler.transform(ea)

                    optimizer.zero_grad()
                    z_pred_ea = model(z_data)

                    ea_pred = y_scaler.inverse_transform(z_pred_ea)
                    stiffness_pred = construct_k_from_ea(ea_pred, nodes, elems, supports, device=device)

                    data_loss = data_criterion(z_pred_ea, z_ea)
                    physics_loss = physics_criterion(stiffness_pred, u, q) / physics_loss_scale

                    loss = data_loss + physics_loss

                    total_loss += loss.item()
                    total_data_loss += data_loss.item()
                    total_physics_loss += physics_loss.item()

            # Log
            val_loss = total_loss / len(val_loader)
            val_physics_loss = total_physics_loss / len(val_loader)
            val_data_loss = total_data_loss / len(val_loader)

            val_losses_fold[fold].append(val_loss)
            val_physics_losses_fold[fold].append(val_physics_loss)
            val_data_losses_fold[fold].append(val_data_loss)

            tb_writer.add_scalar(f'Loss/total/train_FOLD_{fold}', train_loss, epoch)
            tb_writer.add_scalar(f'Loss/total/val_FOLD_{fold}', val_loss, epoch)
            tb_writer.add_scalar(f'Loss/data/train_FOLD_{fold}', train_data_loss, epoch)
            tb_writer.add_scalar(f'Loss/data/val_FOLD_{fold}', val_data_loss, epoch)
            tb_writer.add_scalar(f'Loss/physics/train_FOLD_{fold}', train_physics_loss, epoch)
            tb_writer.add_scalar(f'Loss/physics/val_FOLD_{fold}', val_physics_loss, epoch)

            print(
                f"Fold {fold + 1}, Epoch {epoch + 1}/{N_EPOCHS}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        print(f"Finished fold {fold + 1}")

    train_losses = np.mean(train_losses_fold, axis=0)
    train_data_losses = np.mean(train_data_losses_fold, axis=0)
    train_physics_losses = np.mean(train_physics_losses_fold, axis=0)

    val_losses = np.mean(val_losses_fold, axis=0)
    val_data_losses = np.mean(val_data_losses_fold, axis=0)
    val_physics_losses = np.mean(val_physics_losses_fold, axis=0)

    for i in range(len(train_losses)):
        tb_writer.add_scalar(f'Loss/total/train', train_losses[i], i)
        tb_writer.add_scalar(f'Loss/total/val', val_losses[i], i)
        tb_writer.add_scalar(f'Loss/data/train', train_data_losses[i], i)
        tb_writer.add_scalar(f'Loss/data/val', val_data_losses[i], i)
        tb_writer.add_scalar(f'Loss/physics/train', train_physics_losses[i], i)
        tb_writer.add_scalar(f'Loss/physics/val', val_physics_losses[i], i)

In [ ]:
# Beam 4x60
# Triangle 3x50
# Cantilever 4x50
activation = nn.ReLU
activation_params = {}
lr = 1e-3
dataset_name = 'triangle'
for n_layers in [3, 4]:
    for layer_size in [40, 50, 60]:
        if len(activation_params) == 0:
            writer = SummaryWriter(
                f"./runs/{dataset_name}/FEI_MLP/{n_layers}/{layer_size}/{activation.__name__}/lr_{lr}")
        else:
            writer = SummaryWriter(
                f"./runs/{dataset_name}/FEI_MLP/{n_layers}/{layer_size}/{activation.__name__}/{"".join([f"{k}_{v}" for k, v in activation_params.items()])}/lr_{lr}")

        main(writer, _train_ds[dataset_name], connectivity[dataset_name], support[dataset_name], hidden_size=layer_size,
             n_hidden=n_layers, learning_rate=lr, activation=activation, activation_params=activation_params)

In [20]:
activation = nn.ReLU
activation_params = {}
lr = 1e-3
dataset_name = 'beam'
n_layers = 4
layer_size = 60
for lr in np.logspace(-4, -1, num=5, ):
    if len(activation_params) == 0:
        writer = SummaryWriter(f"./runs/{dataset_name}/FEI_MLP/{n_layers}/{layer_size}/{activation.__name__}/lr_{lr}")
    else:
        writer = SummaryWriter(
            f"./runs/{dataset_name}/FEI_MLP/{n_layers}/{layer_size}/{activation.__name__}/{"".join([f"{k}_{v}" for k, v in activation_params.items()])}/lr_{lr}")

    main(writer, _train_ds[dataset_name], connectivity[dataset_name], support[dataset_name], hidden_size=layer_size,
         n_hidden=n_layers, learning_rate=lr, activation=activation, activation_params=activation_params, n_epochs=150)

Fold 1, Epoch 1/150, Train Loss: 1.9716, Val Loss: 1.9082
Fold 1, Epoch 2/150, Train Loss: 1.8207, Val Loss: 1.7254
Fold 1, Epoch 3/150, Train Loss: 1.6111, Val Loss: 1.5007
Fold 1, Epoch 4/150, Train Loss: 1.4005, Val Loss: 1.3190
Fold 1, Epoch 5/150, Train Loss: 1.2375, Val Loss: 1.1658
Fold 1, Epoch 6/150, Train Loss: 1.0679, Val Loss: 0.9596
Fold 1, Epoch 7/150, Train Loss: 0.8322, Val Loss: 0.7023
Fold 1, Epoch 8/150, Train Loss: 0.5964, Val Loss: 0.4906
Fold 1, Epoch 9/150, Train Loss: 0.4146, Val Loss: 0.3379
Fold 1, Epoch 10/150, Train Loss: 0.2921, Val Loss: 0.2458
Fold 1, Epoch 11/150, Train Loss: 0.2213, Val Loss: 0.1954
Fold 1, Epoch 12/150, Train Loss: 0.1829, Val Loss: 0.1674
Fold 1, Epoch 13/150, Train Loss: 0.1599, Val Loss: 0.1492
Fold 1, Epoch 14/150, Train Loss: 0.1444, Val Loss: 0.1364
Fold 1, Epoch 15/150, Train Loss: 0.1330, Val Loss: 0.1268
Fold 1, Epoch 16/150, Train Loss: 0.1240, Val Loss: 0.1191
Fold 1, Epoch 17/150, Train Loss: 0.1168, Val Loss: 0.1129
Fold 1

KeyboardInterrupt: 

In [24]:
n_layers = 4
layer_size = 40
lr = 4e-4

for activation, activation_params in zip(
        [nn.ReLU, nn.Tanh, nn.Sigmoid, nn.LeakyReLU, nn.LeakyReLU, nn.LeakyReLU],
        [{}, {}, {}, {'negative_slope': 0.01}, {'negative_slope': 0.02}, {'negative_slope': 0.05}]
):
    if len(activation_params) == 0:
        writer = SummaryWriter(f"./runs/{dataset_name}/MLP/{n_layers}/{layer_size}/{activation.__name__}/lr_{lr}")
    else:
        writer = SummaryWriter(
            f"./runs/{dataset_name}/MLP/{n_layers}/{layer_size}/{activation.__name__}/{"".join([f"{k}_{v}" for k, v in activation_params.items()])}/lr_{lr}")

    main(writer, _train_ds[dataset_name], hidden_size=layer_size, n_hidden=n_layers, learning_rate=lr,
         activation=activation, activation_params=activation_params, n_epochs=50)

Fold 1, Epoch 1/50, Train Loss: 0.9698, Val Loss: 0.9121
Fold 1, Epoch 2/50, Train Loss: 0.7999, Val Loss: 0.6349
Fold 1, Epoch 3/50, Train Loss: 0.4544, Val Loss: 0.3158
Fold 1, Epoch 4/50, Train Loss: 0.2541, Val Loss: 0.2030
Fold 1, Epoch 5/50, Train Loss: 0.1708, Val Loss: 0.1469
Fold 1, Epoch 6/50, Train Loss: 0.1302, Val Loss: 0.1210
Fold 1, Epoch 7/50, Train Loss: 0.1105, Val Loss: 0.1054
Fold 1, Epoch 8/50, Train Loss: 0.0969, Val Loss: 0.0934
Fold 1, Epoch 9/50, Train Loss: 0.0861, Val Loss: 0.0836
Fold 1, Epoch 10/50, Train Loss: 0.0773, Val Loss: 0.0770
Fold 1, Epoch 11/50, Train Loss: 0.0705, Val Loss: 0.0722
Fold 1, Epoch 12/50, Train Loss: 0.0649, Val Loss: 0.0652
Fold 1, Epoch 13/50, Train Loss: 0.0604, Val Loss: 0.0630
Fold 1, Epoch 14/50, Train Loss: 0.0567, Val Loss: 0.0593
Fold 1, Epoch 15/50, Train Loss: 0.0532, Val Loss: 0.0529
Fold 1, Epoch 16/50, Train Loss: 0.0500, Val Loss: 0.0534
Fold 1, Epoch 17/50, Train Loss: 0.0473, Val Loss: 0.0499
Fold 1, Epoch 18/50, Tr

Final training

In [27]:
def main_final(
        tb_writer,
        train_dataset,
        test_dataset,
        elems,
        supports,
        out_size=1,
        hidden_size=70,
        n_hidden=3,
        activation=nn.ReLU,
        activation_params={},
        learning_rate=1e-3,
        n_epochs=30,
        save_path=None
):
    in_size = len(train_dataset[0][0])
    N_EPOCHS = n_epochs
    BATCH_SIZE = 2048  # Not an hyperparameter
    LEARNING_RATE = learning_rate

    train_losses = []
    train_data_losses = []
    train_physics_losses = []

    test_losses = []
    test_data_losses = []
    test_physics_losses = []

    # Dataloader

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, )
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, )

    # Training scaler
    x_scaler = StandardScaler(in_size).to(device)
    y_scaler = StandardScaler(out_size).to(device)
    for x, y, _, _, _ in train_loader:
        x_scaler.partial_fit(x.to(device))
        y_scaler.partial_fit(y.to(device))

    # Model setup
    model = MultiLayerPerceptron(in_size, out_size, hidden_size, n_hidden, activation,
                                 activation_params).to(device)

    data_criterion = nn.MSELoss()
    physics_criterion = StiffnessToLoadLoss()

    physics_loss_scale = None

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)
    best_test_loss = float('inf')
    # Model training
    for epoch in range(N_EPOCHS):
        # Train
        model.train()

        total_loss = 0.0
        total_data_loss = 0.0
        total_physics_loss = 0.0

        for batch in train_loader:
            data, ea, nodes, u, q = batch

            data, ea = data.to(device), ea.to(device)
            nodes, u, q = nodes.to(device), u.to(device), q.to(device)

            z_data = x_scaler.transform(data)
            z_ea = y_scaler.transform(ea)

            optimizer.zero_grad()
            z_pred_ea = model(z_data)

            ea_pred = y_scaler.inverse_transform(z_pred_ea)
            stiffness_pred = construct_k_from_ea(ea_pred, nodes, elems, supports, device=device)

            data_loss = data_criterion(z_pred_ea, z_ea)
            physics_loss = physics_criterion(stiffness_pred, u, q)

            if physics_loss_scale is None: physics_loss_scale = physics_loss.item()

            physics_loss /= physics_loss_scale

            loss = data_loss + physics_loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_data_loss += data_loss.item()
            total_physics_loss += physics_loss.item()

        # Log
        train_loss = total_loss / len(train_loader)
        train_physics_loss = total_physics_loss / len(train_loader)
        train_data_loss = total_data_loss / len(train_loader)

        train_losses.append(train_loss)
        train_physics_losses.append(train_physics_loss)
        train_data_losses.append(train_data_loss)

        scheduler.step(train_loss)

        # Validate
        model.eval()
        total_loss = 0.0
        total_data_loss = 0.0
        total_physics_loss = 0.0
        total_MAE = 0.0
        total_MAER = 0.0
        total_MSE = 0.0
        with torch.no_grad():
            for batch in test_loader:
                data, ea, nodes, u, q = batch

                data, ea = data.to(device), ea.to(device)
                nodes, u, q = nodes.to(device), u.to(device), q.to(device)

                z_data = x_scaler.transform(data)
                z_ea = y_scaler.transform(ea)

                optimizer.zero_grad()
                z_pred_ea = model(z_data)

                ea_pred = y_scaler.inverse_transform(z_pred_ea)
                stiffness_pred = construct_k_from_ea(ea_pred, nodes, elems, supports, device=device)

                data_loss = data_criterion(z_pred_ea, z_ea)
                physics_loss = physics_criterion(stiffness_pred, u, q) / physics_loss_scale

                loss = data_loss + physics_loss

                total_loss += loss.item()
                total_data_loss += data_loss.item()
                total_physics_loss += physics_loss.item()

                MAE = torch.abs(ea-ea_pred).mean()
                MAER = torch.abs((ea-ea_pred)/ea).mean()
                MSE = data_criterion(ea, ea_pred)

                total_MAE += MAE.item()
                total_MAER += MAER.item()
                total_MSE += MSE.item()


        # Log
        test_loss = total_loss / len(test_loader)
        test_physics_loss = total_physics_loss / len(test_loader)
        test_data_loss = total_data_loss / len(test_loader)

        test_mse = total_MSE / len(test_loader)
        test_mae = total_MAE / len(test_loader)
        test_maer = total_MAER / len(test_loader)

        test_losses.append(test_loss)
        test_physics_losses.append(test_physics_loss)
        test_data_losses.append(test_data_loss)

        tb_writer.add_scalar(f'Loss/total/train', train_loss, epoch)
        tb_writer.add_scalar(f'Loss/total/test', test_loss, epoch)
        tb_writer.add_scalar(f'Loss/data/train', train_data_loss, epoch)
        tb_writer.add_scalar(f'Loss/data/test', test_data_loss, epoch)
        tb_writer.add_scalar(f'Loss/physics/train', train_physics_loss, epoch)
        tb_writer.add_scalar(f'Loss/physics/test', test_physics_loss, epoch)

        tb_writer.add_scalar(f'Loss/MAE/test', test_mae, epoch)
        tb_writer.add_scalar(f'Loss/MAER/test', test_maer, epoch)
        tb_writer.add_scalar(f'Loss/MSE/test', test_mse, epoch)

        print(
            f"Epoch {epoch + 1}/{N_EPOCHS}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test MAER: {test_maer * 100:.4f} %, Test MAE: {test_mae * 1e-6:.4f} MN, Test MSE: {test_mse * 1e-12:.4f} MN^2",
            end=' ')

        if test_loss < best_test_loss and save_path is not None and epoch > 50:
            best_test_loss = test_loss
            torch.save(model.state_dict(), f"{save_path}/best_model_{test_loss}.pth")
            print("-> MODEL SAVED")
        else:
            print("")


In [28]:
dataset_name = 'beam'
writer = SummaryWriter(f"./runs/final/beam/FEI_MLP/4/60/ReLU/lr_0.0004")
save_path = "./runs/final/beam/FEI_MLP/4/60/ReLU/lr_0.0004/SAVE"
os.makedirs(save_path, exist_ok=True)
model_beam = main_final(writer, _train_ds[dataset_name], _test_ds[dataset_name], connectivity[dataset_name], support[dataset_name], hidden_size=60, n_hidden=4, activation=nn.ReLU,
                        activation_params={}, learning_rate=0.0004, n_epochs=200, save_path=save_path)
writer.close()

/Users/aflamant/miniconda3/envs/memoire/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/200, Train Loss: 1.5929, Test Loss: 1.2005, Test MAER: 69.6500 %, Test MAE: 379.4090 MN, Test MSE: 206471.9044 MN^2 
Epoch 2/200, Train Loss: 0.6806, Test Loss: 0.2310, Test MAER: 26.2230 %, Test MAE: 147.1438 MN, Test MSE: 48211.1430 MN^2 
Epoch 3/200, Train Loss: 0.1603, Test Loss: 0.1302, Test MAER: 18.8771 %, Test MAE: 106.4445 MN, Test MSE: 28280.7455 MN^2 
Epoch 4/200, Train Loss: 0.1158, Test Loss: 0.1050, Test MAER: 16.5011 %, Test MAE: 92.9427 MN, Test MSE: 23160.4766 MN^2 
Epoch 5/200, Train Loss: 0.0938, Test Loss: 0.0870, Test MAER: 14.6564 %, Test MAE: 81.9032 MN, Test MSE: 19444.6719 MN^2 
Epoch 6/200, Train Loss: 0.0777, Test Loss: 0.0717, Test MAER: 12.5897 %, Test MAE: 70.7119 MN, Test MSE: 16166.5147 MN^2 
Epoch 7/200, Train Loss: 0.0659, Test Loss: 0.0604, Test MAER: 11.0212 %, Test MAE: 62.0086 MN, Test MSE: 13704.4182 MN^2 
Epoch 8/200, Train Loss: 0.0562, Test Loss: 0.0523, Test MAER: 9.4743 %, Test MAE: 54.9516 MN, Test MSE: 11898.7221 MN^2 
Epoch 9/200, 

In [31]:
dataset_name = 'triangle'
writer = SummaryWriter(f"./runs/final/{dataset_name}/FEI_MLP/3/50/ReLU/lr_0.0004")
save_path = f"./runs/final/{dataset_name}/FEI_MLP/3/50/ReLU/lr_0.0004/SAVE"
os.makedirs(save_path, exist_ok=True)
model_beam = main_final(writer, _train_ds[dataset_name], _test_ds[dataset_name], connectivity[dataset_name], support[dataset_name], hidden_size=50, n_hidden=3, activation=nn.ReLU,
                        activation_params={}, learning_rate=0.0004, n_epochs=200, save_path=save_path)
writer.close()

Epoch 1/200, Train Loss: 1.5903, Test Loss: 1.1098, Test MAER: 50.1340 %, Test MAE: 471.4121 MN, Test MSE: 438297.1955 MN^2 
Epoch 2/200, Train Loss: 0.6744, Test Loss: 0.3946, Test MAER: 27.4776 %, Test MAE: 273.8418 MN, Test MSE: 186131.0951 MN^2 
Epoch 3/200, Train Loss: 0.2915, Test Loss: 0.2230, Test MAER: 19.3542 %, Test MAE: 197.9738 MN, Test MSE: 114708.1546 MN^2 
Epoch 4/200, Train Loss: 0.1836, Test Loss: 0.1606, Test MAER: 16.2856 %, Test MAE: 163.5788 MN, Test MSE: 85654.6319 MN^2 
Epoch 5/200, Train Loss: 0.1391, Test Loss: 0.1283, Test MAER: 14.3286 %, Test MAE: 142.6707 MN, Test MSE: 69966.4858 MN^2 
Epoch 6/200, Train Loss: 0.1133, Test Loss: 0.1080, Test MAER: 12.9125 %, Test MAE: 128.0267 MN, Test MSE: 59501.9861 MN^2 
Epoch 7/200, Train Loss: 0.0961, Test Loss: 0.0935, Test MAER: 11.5638 %, Test MAE: 115.2963 MN, Test MSE: 51865.9037 MN^2 
Epoch 8/200, Train Loss: 0.0837, Test Loss: 0.0824, Test MAER: 10.3803 %, Test MAE: 104.5032 MN, Test MSE: 45979.8027 MN^2 
Epoch

In [32]:
dataset_name = 'cantilever'
writer = SummaryWriter(f"./runs/final/{dataset_name}/FEI_MLP/4/50/ReLU/lr_0.0004")
save_path = f"./runs/final/{dataset_name}/FEI_MLP/4/50/ReLU/lr_0.0004/SAVE"
os.makedirs(save_path, exist_ok=True)
model_beam = main_final(writer, _train_ds[dataset_name], _test_ds[dataset_name], connectivity[dataset_name], support[dataset_name], hidden_size=50, n_hidden=4, activation=nn.ReLU,
                        activation_params={}, learning_rate=0.0004, n_epochs=200, save_path=save_path)
writer.close()

Epoch 1/200, Train Loss: 1.7966, Test Loss: 1.3356, Test MAER: 87.5679 %, Test MAE: 588.8691 MN, Test MSE: 545909.1587 MN^2 
Epoch 2/200, Train Loss: 0.8347, Test Loss: 0.4419, Test MAER: 44.0312 %, Test MAE: 327.0296 MN, Test MSE: 219770.1872 MN^2 
Epoch 3/200, Train Loss: 0.3267, Test Loss: 0.2414, Test MAER: 33.1893 %, Test MAE: 241.1810 MN, Test MSE: 128136.5067 MN^2 
Epoch 4/200, Train Loss: 0.2018, Test Loss: 0.1702, Test MAER: 26.1439 %, Test MAE: 197.4001 MN, Test MSE: 93019.8182 MN^2 
Epoch 5/200, Train Loss: 0.1503, Test Loss: 0.1304, Test MAER: 21.9692 %, Test MAE: 168.9178 MN, Test MSE: 73218.7702 MN^2 
Epoch 6/200, Train Loss: 0.1171, Test Loss: 0.1034, Test MAER: 18.6759 %, Test MAE: 144.5013 MN, Test MSE: 59087.9315 MN^2 
Epoch 7/200, Train Loss: 0.0933, Test Loss: 0.0837, Test MAER: 16.2726 %, Test MAE: 124.4734 MN, Test MSE: 48352.1425 MN^2 
Epoch 8/200, Train Loss: 0.0768, Test Loss: 0.0693, Test MAER: 14.7933 %, Test MAE: 110.1348 MN, Test MSE: 40226.7248 MN^2 
Epoch